In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from lsst.daf.butlerUtils import ExposureIdInfo

import lsst.afw.image              as afwImage
import lsst.afw.geom               as afwGeom
import lsst.afw.table              as afwTable

import lsst.meas.algorithms        as measAlg

Load the high-level "tasks" that process the pixels

In [ ]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate         import CalibrateTask
from lsst.meas.algorithms.detection    import SourceDetectionTask
from lsst.meas.deblender               import SourceDeblendTask
from lsst.meas.base                    import SingleFrameMeasurementTask

Load a version of psfex that implements our Psf object

In [ ]:
import lsst.meas.extensions.psfex.psfexPsfDeterminer

The next import will fail until you've explicitly built and setup the Kron extension

    git clone git@github.com:lsst/meas_extensions_photometryKron
    setup -r . -j
    scons -Q opt=3 -j 4
    
and then restarted your jupyter notebook

In [ ]:
try:
    import lsst.meas.extensions.photometryKron
    doKron = True
except ImportError as e:
    doKron = False

Setup the displays (by default an interface to ds9)

In [ ]:
disp  = afwDisplay.Display(1)
disp2 = afwDisplay.Display(2)

## Create the tasks

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()

config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
#config.measurePsf.psfDeterminer.name = "pca"
charImageTask =         CharacterizeImageTask(config=config)

config = SourceDetectionTask.ConfigClass()
#config.thresholdValue = 30
#config.doTempLocalBackground = True
sourceDetectionTask =   SourceDetectionTask(schema=schema, config=config)

sourceDeblendTask =     SourceDeblendTask(schema=schema)

config = SingleFrameMeasurementTask.ConfigClass()
config.doApplyApCorr = 'yes'
config.slots.apFlux = 'base_CircularApertureFlux_12_0'
if doKron:
    config.plugins.names.add("ext_photometryKron_KronFlux")
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config,
                                                   algMetadata=algMetadata)

It is a bug that I need to add this field to the metadata

In [ ]:
if doKron:
    algMetadata.set("ext_photometryKron_KronFlux_nRadiusForFlux",
                    config.plugins["ext_photometryKron_KronFlux"].nRadiusForFlux)

## Time to process some data

#### Read the input data

In [ ]:
fileName = "example1.fits"
exposureIdInfo = ExposureIdInfo(0, 5)

exposure = afwImage.ExposureF(fileName)

#### Create the output table

In [ ]:
tab = afwTable.SourceTable.make(schema)

#### Process the pixels

##### Characterise the exposure (e.g. estimate the PSF)

In [ ]:
result = charImageTask.characterize(exposure, exposureIdInfo)

psfCellSet = result.psfCellSet     # we'll look at this data structure later

##### Detect objects (`sources')

In [ ]:
result = sourceDetectionTask.run(tab, exposure)
sources = result.sources

##### Deblend overlapping objects

In [ ]:
sourceDeblendTask.run(exposure, sources, exposure.getPsf())

##### Measure the objects' properties

In [ ]:
sourceMeasurementTask.run(exposure, sources)

#### Write the results to a FITS file (if desired)

In [ ]:
fitsTable = "outputTable.fits"
sources.writeFits(fitsTable)

exposure.writeFits("example1-out.fits")

## OK, we've finished the image processing.

We have our list of sources, but it's not currently continuous in memory

In [ ]:
try:
    sources.get("id")
except Exception as e:
    print e

We can fix this with a deep copy

In [ ]:
sources = sources.copy(True)

In [ ]:
if False:
    sources = afwTable.SourceCatalog.readFits(fitsTable)

## Look at the results

In [ ]:
import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.image.utils as afwImageUtils

Define a boolean array that tells us which objects are 'good'; in this case:
 - No saturated pixels near their centres
 - terminal objects that haven't been further deblended

In [ ]:
good = np.logical_and.reduce([sources.get('base_PixelFlags_flag_saturatedCenter') == 0,
                              sources.get("deblend_nChild") == 0,
                              ])

#### Look at how well the aperture (maybe including Kron) photometry agrees with the PSF measurements

In [ ]:
with afwImageUtils.CalibNoThrow():
    apMag = exposure.getCalib().getMagnitude(sources.getApFlux())
    #apMag = exposure.getCalib().getMagnitude(sources["base_CircularApertureFlux_12_0_flux"])
    if doKron:
        kronMag = exposure.getCalib().getMagnitude(sources["ext_photometryKron_KronFlux_flux"])
    psfMag = exposure.getCalib().getMagnitude(sources.getPsfFlux())

for mag, name in [
                    (apMag, "aperture"),
                    #(kronMag, "Kron")
                 ]:
    plt.plot(apMag[good], (psfMag - mag)[good], 'o', label=name)

plt.legend(loc='best')
plt.axhline(0.0, ls=':', color='black')
plt.xlim(14, 24.5)
plt.ylim(-0.2, 0.8)

plt.xlabel("apMag")
plt.ylabel("psfMag - mag")

plt.show()

This doesn't look like modern CCD data as there's no brighter-fatter effect.  That's because we corrected it at the pixel level as part of the instrumental signature effect.

In [ ]:
if not False:
    disp2.mtv(exposure, title='post calib')
else:
    disp2.erase()

#### Show the PSF candidates on the image

In [ ]:
if psfCellSet:
    with disp2.Buffering():
        for cell in psfCellSet.getCellList():
            afwDisplayUtils.drawBBox(cell.getBBox(), display=disp2, 
                                     borderWidth=0.5, ctype=afwDisplay.CYAN)

            for cand in cell:
                disp2.dot("*", cand.getXCenter(), cand.getYCenter(),
                          ctype=afwDisplay.GREEN if cand.GOOD else afwDisplay.RED)

##### Look at the residuals from the PSF modelling

In [ ]:
residuals = exposure.getMaskedImage().getImage().clone()
psf = exposure.getPsf()

with disp.Buffering():
    for s in sources[good]:
        psfImage = psf.computeImage(s.getCentroid()).convertF()
        psfImage *= s.getPsfFlux()/np.sum(psfImage.getArray())
            
        try:
            residuals[psfImage.getBBox()] -= psfImage
        except:
            pass
            
disp.mtv(residuals)

with disp.Buffering():
    for s in sources[good]:
        disp.dot("o", *s.getCentroid(), size=20) 

#### Now show something about the measurements

In [ ]:
disp2.erase()

In [ ]:
with disp2.Buffering():
    for s in sources[good]:
        disp2.dot('+', *s.getCentroid(),
                    ctype=afwDisplay.CYAN if s.get("flags_negative") else afwDisplay.GREEN)

In [ ]:
if doKron:
    disp2.erase()

    Kron_nRadiusForFlux = algMetadata.get("ext_photometryKron_KronFlux_nRadiusForFlux")
    with disp2.Buffering():
        for s in sources[good]:
            shape = s.getShape().clone()
            rDet = shape.getDeterminantRadius()

            shape.scale(s["ext_photometryKron_KronFlux_radius"]/rDet)
            disp2.dot(shape, *s.getCentroid(), ctype=afwDisplay.RED)
            
            shape.scale(Kron_nRadiusForFlux)
            disp2.dot(shape, *s.getCentroid(), ctype=afwDisplay.CYAN)

In [ ]:
if False:
    radii = algMetadata.get("base_CircularApertureFlux_radii")

    with disp2.Buffering():
        for s in sources[good]:
            for radius in radii:
                disp2.dot('o', *s.getCentroid(), size=radius, ctype=afwDisplay.YELLOW)